In [1]:
# 1. 랭스미스 초기화

import os
from dotenv import load_dotenv
from langchain.callbacks.manager import tracing_v2_enabled

load_dotenv()

# 트레이싱 상태 확인
with tracing_v2_enabled() as tracing:
    print(f"Tracing enabled: {tracing}")


Tracing enabled: <langchain_core.tracers.langchain.LangChainTracer object at 0x10c853950>


In [2]:

# 2. 체인 초기화 및 실행
from llm_compiler import initialize_chain
from langchain_core.messages import HumanMessage
from pprint import pprint

chain = initialize_chain()

# 3. 스트리밍 실행
for step in chain.stream(
    {"messages": [HumanMessage(content="한화리츠가 발표한 운용수익은 직전 보고 기간 대비 얼마나 변동되었는지 궁금합니다.")]}
):
    print(step)
    print("---")

print("---------------------최종 답변 --------------------------")
print("------------------------------------------------------")
pprint(step["join"]["messages"][-1].content)
print("------------------------------------------------------")
print("------------------------------------------------------")

캐시된 순매수 가중치 사용
기존 벡터스토어를 로드합니다...
기존 기업 벡터스토어를 로드합니다...

=== 기업명 추출 시작 ===
입력 쿼리: 한화리츠가 발표한 운용수익은 직전 보고 기간 대비 얼마나 변동되었는지 궁금합니다.

[Step 1: LLM으로 회사명 텍스트 추출 중...]
LLM 응답: content='["한화리츠"]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 484, 'total_tokens': 493, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-60a3a723-3b73-40d6-b9b7-391a77f3fdb2-0' usage_metadata={'input_tokens': 484, 'output_tokens': 9, 'total_tokens': 493, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
- LLM이 예상한 회사명 목록: ['한화리츠']

[Step 2: 한화리츠에 대한 유사 기업명 후보 추출 중...]


ERROR:root:plan_and_schedule 전체 실행 중 예기치 않은 오류: min() arg is an empty sequence


{'한화리츠': 1.0, '한다리츠': 0.75, '한국리츠': 0.75, '한경리츠': 0.75, '하우리츠': 0.725, '한화': 0.6, '한화포리마': 0.6, '서명리츠': 0.5, '플리츠': 0.5, '경성리츠': 0.5, '동방리츠': 0.5, '한미리': 0.5, '드림리츠': 0.5, '파일리츠': 0.5, '브이리츠': 0.5, '한국리텍': 0.5, '평안리츠': 0.5, '엘리츠': 0.5, '한화비전': 0.5, '프리츠': 0.5, '코리츠': 0.5, '롯데리츠': 0.5, '한화엔진': 0.5, '한누리': 0.5, '에이리츠': 0.5, '한화미': 0.5, '한화생명': 0.5, '한화갤러리아': 0.5, '한얼리치': 0.5, '조일리츠': 0.5, '신한알파리츠': 0.5, '한화상역': 0.5, '중부리츠': 0.5, '한화건설': 0.5, '케이리츠': 0.5, '한화포리': 0.5, '한화오션': 0.5, '한국리복': 0.5, '호텔리츠': 0.5, '한화제약': 0.5, '한화방산': 0.5, '대한리츠': 0.5, '무등리츠': 0.5, '한화개발': 0.5, '한림': 0.475, '릿츠': 0.475, '한황산업': 0.475, '화림': 0.475, '한국릴리': 0.475, '한립': 0.475}
{'plan_and_schedule': {'messages': [], 'replan_count': 0, 'task_results': []}}
---
stop_joiner 값은 뭐야 ?  False
chain 씀
refference 확인 *************
{'join': {'messages': [AIMessage(content="Thought: The user's question requires specific financial data about Hanwha REITs' operational income changes compared to the previous reporting period. Unfo

ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


(resolved 전)*args check *************
{'query': '한화리츠 운용수익', 'company': '한화리츠', 'year': 2024}

Resolving dict args: {'query': '한화리츠 운용수익', 'company': '한화리츠', 'year': 2024}
******************************
*Whatis my arg ? *************
한화리츠 운용수익
*Observations check *************
{}
******************************
*Whatis my arg ? *************
한화리츠
*Observations check *************
{}
******************************
*Whatis my arg ? *************
2024
*Observations check *************
{}
tool에 진짜 들어가는 형태의 args
{'query': '한화리츠 운용수익', 'company': '한화리츠', 'year': 2024}
observations {}
tool에 진짜 들어가는 형태의 args
{'query': '한화리츠 운용수익', 'company': '한화리츠', 'year': 2024}
입력 회사명 : 한화리츠, 입력 연도 : 2024
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

사업 보고서 번호:  Non

ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


observations
{'plan_and_schedule': {'messages': [FunctionMessage(content="{'output': '한화리츠의 2024년도 운용수익에 대한 정보는 다음과 같습니다:\\n\\n- **운용수익 정의**: 운용수익은 임대료 수익, 관리 수익, 주차 수익 및 기타 수익을 합친 금액을 의미합니다.\\n- **2023년 기준 당기 운용수익**: 22,808,550원\\n- **2023년 전기 운용수익**: 23,274,064원\\n\\n2024년도의 구체적인 운용수익 정보는 제공되지 않았으나, 2023년 기준으로는 전기 대비 감소한 수익을 기록하였습니다.\\n\\n출처: [한화리츠 사업보고서](https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20240722000186&dcm_no=10034163) (페이지 59)', 'key_information': [{'tool': '사업보고서', 'referenced_content': '운용수익은 임대료수익, 관리수익, 주차수익 및 기타수익을 합친 금액입니다.', 'page_number': 59, 'filename': '한화리츠/20240722000186/서비스업/[III. 재무에 관한 사항]', 'link': 'https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20240722000186&dcm_no=10034163'}, {'tool': '사업보고서', 'referenced_content': '당기 운용수익: 22,808,550(전기 23,274,064), 출처: 문서 내용', 'page_number': 59, 'filename': '한화리츠/20240722000186/서비스업/[III. 재무에 관한 사항]', 'link': 'https://dart.fss.or.kr/pdf/download/pdf.do?rcp_no=20240722000186&dcm_no=10034163'}]}", additional_k